[View in Colaboratory](https://colab.research.google.com/github/Curiousss/FLD/blob/master/FLDNetwork.ipynb)

In [1]:
!wget https://www.dropbox.com/s/9j2om0gcv43hkz8/300w_cropped.zip


Redirecting output to ‘wget-log.2’.
^C


In [0]:
!wget https://www.dropbox.com/s/o3615gx23xohs00/menpo_challenge_trainset.zip

In [0]:
!unzip 300w_cropped.zip

In [22]:
!unzip menpo_challenge_trainset.zip

Archive:  menpo_challenge_trainset.zip
   creating: menpo_train_release/
[menpo_challenge_trainset.zip] menpo_train_release/aflw__face_39822.jpg password: 

In [8]:
!ls

300w_cropped.zip  300w_cropped.zip.1  sample_data  wget-log  wget-log.1


In [6]:
from google.colab import files
uploaded = files.upload()

Saving pts_loader.py to pts_loader.py


In [6]:
!git clone https://github.com/albanie/pts_loader
!ls pts_loader/
!cp pts_loader/pts_loader.py .
!ls

Cloning into 'pts_loader'...
fatal: unable to access 'https://github.com/albanie/pts_loader/': gnutls_handshake() failed: Error in the pull function.
ls: cannot access 'pts_loader/': No such file or directory
cp: cannot stat 'pts_loader/pts_loader.py': No such file or directory
01_Indoor  02_Outdoor  300w_cropped.zip  kernel45.txt  sample_data  wget-log


In [3]:
import os
from os import listdir,makedirs
from os.path import isfile,join
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import cv2

import tensorflow as tf
import keras.backend as K
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Add, SeparableConv2D
from keras.layers import Convolution2D, Lambda, Input, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.layers import LeakyReLU
from keras import optimizers




Using TensorFlow backend.


In [0]:
'''
Hyper-Parameters
'''
image_H = 96.0
image_W = 96.0
grayscale = True
batch_size = 16
learning_rate=0.001
momentum = 0.9
decay = 0
epochs = 100
droupout= 0.1
gradient_clipping = 0.1
nFilters = 128
nlandmarks = nLabels = 68
K=45
img_rows = 96
img_cols = 96
img_channels = 1
alpha = 0.1
epsilon = 0.00001

In [0]:
# Get files
import pts_loader

img_x = []
pts_y = []
for i in range(1,600):
    #print("iteration", i)
    if i == 78 or i == 79 or i == 266 or i == 385 or i == 478 or i == 549 or i == 550:
      continue
    if i < 10:
        img_f = r"01_Indoor/indoor_00" + str(i) +".png"
        pts_f = r"01_Indoor/indoor_00" + str(i) +".pts"
    elif i < 100:
        img_f = r"01_Indoor/indoor_0" + str(i) +".png"
        pts_f = r"01_Indoor/indoor_0" + str(i) +".pts"
    elif i < 301:
        img_f = r"01_Indoor/indoor_" + str(i) +".png"
        pts_f = r"01_Indoor/indoor_" + str(i) +".pts"
    
    elif i < 310:
        j = i - 300
        img_f = r"02_Outdoor/outdoor_00" + str(j) +".png"
        pts_f = r"02_Outdoor/outdoor_00" + str(j) +".pts"
    elif i < 400:
        j = i - 300
        img_f = r"02_Outdoor/outdoor_0" + str(j) +".png"
        pts_f = r"02_Outdoor/outdoor_0" + str(j) +".pts"
    else:
        j = i - 300    
        img_f = r"02_Outdoor/outdoor_" + str(j) +".png"
        pts_f = r"02_Outdoor/outdoor_" + str(j) +".pts"
    pts = pts_pts = pts_loader.load(pts_f)
    # Loading imgae in grayscale
    img = cv2.imread(img_f)
    
    scale_x = image_W/img.shape[0]
    scale_y = image_H/img.shape[1]
    img = cv2.resize(img, (img_rows, img_cols))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.normalize(img,None,0,1,cv2.NORM_MINMAX)
    img = img.reshape(img_rows, img_cols, 1)
    landmarks = []
    landmarks1 = []
    
    for p in pts:
        new_pts = []
        landmark = np.zeros((96, 96), np.uint8)
        new_pts = (p[0] * scale_y , p[1] * scale_x )
        #new_points.append(new_pts)
        x = int(new_pts[0])
        y = int(new_pts[1])
        #print(n, x, y)
        landmark[x][y] = 1 # Normalized WHite
        landmark1 = cv2.flip(landmark,1) # Augementation: Horizontal flip
        landmarks.append(landmark)
        landmarks1.append(landmark1) # Augementation: Horizontal flip
        #landmarks.append(x) # Testing
        #landmarks.append(y) # Testing
        
    img1 = cv2.flip(img,1) # Augementation: Horizontal flip
    img1 = img1.reshape(img_rows, img_cols, 1)
    img_x.append(img)
    pts_y.append(landmarks)
    img_x.append(img1)
    pts_y.append(landmarks1)


In [5]:
print(img_x[1].shape)
assert not np.any(np.isnan(img_x))
assert not np.any(np.isnan(pts_y))
train_x = []
train_y = []
test_x = []
test_y = []


(96, 96, 1)


In [0]:
train_x = np.array(img_x[:1000])
train_y = np.array(pts_y[:1000])
test_x = np.array(img_x[1000:])
test_y = np.array(pts_y[1000:])


In [7]:
print(train_y.shape)
train_y = train_y.reshape(-1, 96,96,68)
test_y = test_y.reshape(-1, 96,96,68)

print(train_y.shape)
print(test_y.shape)

(1000, 68, 96, 96)
(1000, 96, 96, 68)
(184, 96, 96, 68)


In [0]:
def apply_kernel(local_out):
    '''
     get kernel from text file
     tile it to 1x1x45x45
     convolve with local out
    '''
    f = open(r'kernel45.txt')
    kernel45 = np.fromfile(f, dtype = np.float32, count = -1, sep = " ")
    
    # [1 x 1 x 45 x 45]
    kernel45 = np.reshape(kernel45, (45, 45, 1, 1))
    # [numLabels x 1 x 45 x 45] if tensor convolution is used
    kernel_stacked 	= np.tile(kernel45, (1, 1, nLabels, 1))
    kernel_tensor = tf.convert_to_tensor(kernel_stacked, dtype=tf.float32)
    print("local_out", local_out)
    print("kernel", kernel_tensor)

    return tf.nn.depthwise_conv2d(local_out, kernel_tensor, strides=[1,1,1,1],
                                   padding="SAME")




In [0]:
def FLD_model_build():
  '''
  ----------------------------------------LOCAL SUBNET----------------------------
  '''

  # [ImageC x ImageH x ImageW]
  #-----------------------------------------------5 layers of kernel 5x5----------
  input_img = Input(( img_rows, img_cols, img_channels))
  model = SeparableConv2D(filters=nFilters, kernel_size=(5, 5), padding='same')(input_img)
  model = BatchNormalization(epsilon=epsilon)(model)
  model = LeakyReLU(alpha)(model)
  model = Dropout(droupout)(model)

  for i in range(4):
    model = SeparableConv2D(filters=nFilters, kernel_size=(5, 5), padding='same')(model)
    model = BatchNormalization(epsilon=epsilon)(model)
    model = LeakyReLU(alpha)(model)
    model = Dropout(droupout)(model)

  #-----------------------------------------------10 layers of kernel 3x3----------
  #1
  for i in range(10):
    model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same')(model)
    model = BatchNormalization(epsilon=epsilon)(model)
    model = LeakyReLU(alpha)(model)
    model = Dropout(droupout)(model)

  print("Local subnet", model)

  '''
  ----------------------1x1 Convolution with activation-----------------------------
  Apply Gaussian Kernel filter
  '''
  model_Conv = Convolution2D(filters=nLabels, kernel_size=(1, 1))(model)
  model = LeakyReLU(alpha)(model)
  model_Conv = Lambda(apply_kernel, output_shape=(img_cols, img_rows, nLabels))(model_Conv)
  print("Main 1x1", model_Conv)
  '''
  -------------------------------------Ol 1x1 Linear convolution-------------------
  Apply Gaussian Kernel filter
  '''
  model_Ol = Convolution2D(filters=nLabels, kernel_size=(1, 1),activation=None)(model)
  model_Ol = Lambda(apply_kernel, output_shape=(img_cols, img_rows, nLabels), name='Ol')(model_Ol)
  print("Local output", model_Ol)

  output_Ol = keras.layers.concatenate([model_Conv, model_Ol])
  print("Concatenate", output_Ol)
  output_Ol = Reshape((96, 96, -1))(output_Ol)
  #print("Reshape", output_Ol)

  '''
  ----------------------------------------GLOBAL SUBNET----------------------------
  '''
  #--------------------------------7 layers of dialated kernel 3x3------------------
  #1
  model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4)(output_Ol)
  model = BatchNormalization(epsilon=epsilon)(model)
  model = LeakyReLU(alpha)(model)
  model = Dropout(droupout)(model)
  #2
  for i in range(6):
    model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4)(model)
    model = BatchNormalization(epsilon=epsilon)(model)
    model = LeakyReLU(alpha)(model)
    model = Dropout(droupout)(model)

  model = Convolution2D(filters=nLabels, kernel_size=(1, 1), activation=None)(model)

  FLD_model = Model(inputs = input_img, outputs=model)
  #FLD_model.summary()
  return FLD_model, model_Ol


In [0]:
def landmark_weight(labels):
    
    #[numLabels x 1 x 1]
    W1 = tf.reduce_sum(labels, axis=1)
    W2 = tf.reduce_sum(W1, axis=2)
    W3 = tf.reduce_mean(W2)
    W4 = tf.multiply(W3, 0.5)
    Win = tf.add(W4, W2)
    #Win = tf.reduce_sum(Win)
    print("Win", Win.shape)
    #Win = tf.reshape(Win, (-1, 1, 1, nLabels))
    return Win

'''
Ol - [numLabels x ImageH x ImageW] Kernel conv already applied in Network
Og - [numLabels x ImageH x ImageW]
labels - Landmark coordinates [numLabels x ImageH x ImageW] without
kernel applied
'''
def loss_function(Ol):
  def FLD_loss_func(y_true, y_pred):
      labels = y_true
      Og = y_pred

      # Ground Truth Heatmap
      Gn = apply_kernel(labels)

      Win = landmark_weight(labels)
      global_diff = tf.subtract(Og, Gn)
      global_err = tf.square(global_diff)

      # (Ol - Gn)2

      # Kernel is already applied for Ol
      local_diff = tf.subtract(Ol, Gn)
      local_err = tf.square(local_diff)
      
      comb_err = tf.add(local_err, global_err)
      
      comb_Win = tf.multiply(tf.transpose(Win), tf.transpose(comb_err))
      wt_square_err = tf.reduce_sum(comb_Win)
      wt_square_err = tf.multiply(wt_square_err, 0.05)
      print("Loss", wt_square_err)
      return wt_square_err    
      
      #return tf.reduce_sum(comb_err) #testing
  return FLD_loss_func



In [19]:
FLD_model, model_Ol = FLD_model_build()
sgd = optimizers.SGD(lr=0.001, decay=0, momentum=momentum, nesterov=True, clipnorm=0.1)
loss1 = loss_function(model_Ol)
FLD_model.compile(optimizer=sgd,
              loss=[loss1],
              metrics=['accuracy'])


Local subnet Tensor("dropout_59/cond/Merge:0", shape=(?, 96, 96, 128), dtype=float32)
local_out Tensor("conv2d_7/BiasAdd:0", shape=(?, 96, 96, 68), dtype=float32)
kernel Tensor("lambda_3/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Main 1x1 Tensor("lambda_3/depthwise:0", shape=(?, 96, 96, 68), dtype=float32)
local_out Tensor("conv2d_8/BiasAdd:0", shape=(?, 96, 96, 68), dtype=float32)
kernel Tensor("Ol_2/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Local output Tensor("Ol_2/depthwise:0", shape=(?, 96, 96, 68), dtype=float32)
Concatenate Tensor("concatenate_3/concat:0", shape=(?, 96, 96, 136), dtype=float32)
local_out Tensor("conv2d_9_target:0", shape=(?, ?, ?, ?), dtype=float32)
kernel Tensor("loss_2/conv2d_9_loss/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Win (?, ?)
Loss Tensor("loss_2/conv2d_9_loss/Mul_2:0", shape=(), dtype=float32)


In [22]:
 FLD_model.fit(train_x, train_y, batch_size=8, epochs=10, validation_data=(test_x, test_y))

Train on 1000 samples, validate on 184 samples
Epoch 1/10
1000/1000 [==============================] - 362s 362ms/step - loss: 4024855.4800 - acc: 0.0245 - val_loss: 10452666.3696 - val_acc: 0.0037
Epoch 2/10
1000/1000 [==============================] - 363s 363ms/step - loss: 3912370.1200 - acc: 0.0244 - val_loss: 8750468.4783 - val_acc: 0.0054
Epoch 3/10
1000/1000 [==============================] - 362s 362ms/step - loss: 3840404.7480 - acc: 0.0244 - val_loss: 6839273.8913 - val_acc: 0.0043
Epoch 4/10
1000/1000 [==============================] - 362s 362ms/step - loss: 3775294.4960 - acc: 0.0245 - val_loss: 6124096.2174 - val_acc: 0.0050
Epoch 5/10
1000/1000 [==============================] - 363s 363ms/step - loss: 3727730.5160 - acc: 0.0245 - val_loss: 5512480.3913 - val_acc: 0.0051
Epoch 6/10
1000/1000 [==============================] - 363s 363ms/step - loss: 3678295.3480 - acc: 0.0245 - val_loss: 4963945.1957 - val_acc: 0.0040
Epoch 7/10
1000/1000 [==============================

In [0]:
# Prediction and visualization of landmarks on Image
img = []#get image
scale_x = image_W/img.shape[0]
scale_y = image_H/img.shape[1]

img1 = cv2.resize(img, (img_rows, img_cols))
img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img1 = cv2.normalize(img1,None,0,1,cv2.NORM_MINMAX)
pred = FLD_model.predict(img1)
landmarks = []
for p in pred:
  landmark_x, landmark_y = p.argmax()#get the position
  landmark_x = landmark_x / scale_x
  landmark_y = landmark_y / scale_y
  landmarks.append((landmark_x, landmark_y))

  # Mark scaled landmarks in original image
  cv2.circle(img, (round(landmark_x), round(landmark_y)),  2, (255,255,255), -1)

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [0]:
import cntk as C

cntk_model = C.load_model(r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\Merget\Merget\python\output\models\model.21")

In [0]:
cntk_model.parameters

In [0]:
cntk_model.save("cntk_model.h5")

In [0]:
FLD = keras.models.load_model(r"C:\Users\Meers\Documents\Techie\InkersIntern\Phase2\FLDCode\cntk_model.h5")

In [0]:
import keras.backend as K

#save_model_hdf5(cntk_model, 'my_model.h5')
kerasmodel = K.functional_ops. load_model(cntk_model)

In [0]:
#func = cntk_model.outputs[0]
cntk_model.save('cntk_model.onnx', format=C.ModelFormat.)

In [0]:
import onnx
onnx_model = onnx.load('cntk_model.onnx')

In [0]:
from onnx_tf.backend import prepare
tf_model = prepare(onnx_model)